In [1]:
pip install pyspark==3.3.1

     |████████████████████████████████| 281.4 MB 11 kB/s s eta 0:00:01   |█▎                              | 11.3 MB 2.7 MB/s eta 0:01:40     |██▍                             | 21.4 MB 6.0 MB/s eta 0:00:44     |███                             | 27.1 MB 4.0 MB/s eta 0:01:04     |███▌                            | 31.2 MB 3.8 MB/s eta 0:01:07     |██████                          | 53.0 MB 1.5 MB/s eta 0:02:32     |██████▍                         | 56.1 MB 1.5 MB/s eta 0:02:30     |██████▍                         | 56.3 MB 1.5 MB/s eta 0:02:30     |███████▍                        | 64.6 MB 2.9 MB/s eta 0:01:14     |██████████▌                     | 92.5 MB 11.8 MB/s eta 0:00:17     |████████████▏                   | 106.7 MB 4.1 MB/s eta 0:00:43     |██████████████                  | 122.8 MB 14.0 MB/s eta 0:00:12     |██████████████▌                 | 127.6 MB 14.0 MB/s eta 0:00:11     |███████████████                 | 132.8 MB 14.0 MB/s eta 0:00:11     |███████████████▌                | 

In [2]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession\
    .builder\
    .appName("twitter_transformation")\
    .getOrCreate()

23/01/16 15:01:11 WARN Utils: Your hostname, river-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.133.128 instead (on interface ens33)
23/01/16 15:01:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/16 15:01:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
df = spark.read.json("../../../datalake/twitter_datascience")

In [8]:
df.show()

+--------------------+--------------------+--------------------+------------+
|                data|            includes|                meta|extract_date|
+--------------------+--------------------+--------------------+------------+
|[{737142202481016...|{[{2016-05-30T04:...|{1614960801781252...|  2023-01-16|
|[{156551075009305...|{[{2022-09-02T01:...|{1614960439531819...|  2023-01-16|
|[{161308754706814...|{[{2023-01-11T08:...|{1614960279242276...|  2023-01-16|
|[{109402247573690...|{[{2019-02-08T23:...|{1614960021451976...|  2023-01-16|
|[{101181765595789...|{[{2018-06-27T03:...|{1614959798633795...|  2023-01-16|
|[{137155121135112...|{[{2021-03-15T19:...|{1614959535180898...|  2023-01-16|
|[{142970475005707...|{[{2021-08-23T07:...|{1614959337641762...|  2023-01-16|
|[{18570502, 16149...|{[{2009-01-03T02:...|{1614959152417079...|  2023-01-16|
|[{161308682069857...|{[{2023-01-11T08:...|{1614958997207154...|  2023-01-16|
|[{130630626775391...|{[{2020-09-16T18:...|{1614958862225838...|